<a href="https://colab.research.google.com/github/basetenlabs/demos/blob/main/Deploying%20a%20LightGBM%20Model%20with%20Baseten.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Baseten](https://assets.website-files.com/624535121db2930bcd043f5d/62453d9bddc3de287134cb76_baseten-logo.svg)

# Install dependencies

In [ ]:
!pip install lightgbm
!pip install scikit-learn
!pip install baseten

# Load wine data and split intro training and testing datasets

In [ ]:
import joblib
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
X, y = load_wine(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1010)

# Train LightGBM model

In [ ]:
import lightgbm as lgb

model = lgb.LGBMClassifier(
    learning_rate=0.09,
    max_depth=-5,
    random_state=1010
)
model.fit(
    X_train, 
    y_train, 
    eval_set=[(X_test, y_test), (X_train ,y_train)],
    verbose=20,
    eval_metric='logloss'
)

# Serialize model files for deployment

In [ ]:
with open("model.joblib", "wb") as f:
   joblib.dump(model, f)

# a simple model server for this LGBMClassifier
MODEL_CODE_AS_STR = """import joblib

class LGBModelServer(object):
    def load(self):
        self.model = joblib.load("model/model.joblib")
    def predict(self, inputs):
        class_predictions = self.model.predict(inputs)
        return class_predictions
"""

with open('lgb_model.py', 'w') as py_file:
    py_file.write(MODEL_CODE_AS_STR)

# our only requirement is this package
!pip freeze | egrep 'lightgbm|joblib' > requirements.txt

# Call Baseten API

In [ ]:
import baseten
baseten.login("*** INSERT API KEY ***") # https://docs.baseten.co/applications/overview/api-keys

baseten.deploy_custom(
    model_name='Light Gradient Boosted Wine classifier model',
    model_class='LGBModelServer',
    model_files=['lgb_model.py', 'model.joblib'],
    requirements_file='requirements.txt'
)